<font size = "13px;">Kaggle Competition - [MLB Forecasting](https://www.kaggle.com/c/mlb-player-digital-engagement-forecasting/data)</font>

**Binary columns** will have null values as well as zeroes. <br>
**Zeroes** will occur if a player had an opportunity to do something, but did not.<br>
**Nulls** will occur if a player never had the opportunity to do something (for example: a player who does not pitch on a given day cannot possibly pitch a shutout - therefore a null value is expected)

# Data Loading

In [8]:
import sys
import csv

csv.field_size_limit(100000000)

100000000

In [9]:
from google.colab import drive
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

drive.mount('/content/gdrive')

filenames = ['teams', 'seasons', 'players', 'awards']
path = "/content/gdrive/My Drive/Colab Notebooks/MLB/"

for x in range(0, 6):
    globals()['train%s' % x] = pd.read_csv(path + "train_%s.csv" % x, engine = 'python', error_bad_lines=False)

for filename in filenames:
    globals()['%s' % filename] = pd.read_csv(path + "%s.csv" % filename, engine = 'python', error_bad_lines=False)

example_test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/MLB/example_test.csv')
example_sample_submission = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/MLB/example_sample_submission.csv')

train = pd.concat([train0, train1, train2, train3, train4, train5], axis = 0)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
import ipywidgets as widgets

kaggle_data_tabs = widgets.Tab()
kaggle_data_tabs.children = list([widgets.Output() for filename in filenames])

for index in range(0, len(filenames)):
    
    kaggle_data_tabs.set_title(index, filenames[index])
    
    with kaggle_data_tabs.children[index]:
        display(eval(filenames[index]))

display(kaggle_data_tabs)

# EDA

# Preprocessing

In [11]:
train.isna().sum()

Unnamed: 0                    0
date                          0
nextDayPlayerEngagement       0
games                       577
rosters                       0
playerBoxScores             678
teamBoxScores               678
transactions                112
standings                   685
awards                      919
events                      680
playerTwitterFollowers     1172
teamTwitterFollowers       1172
dtype: int64

## JSON Parsing (reffering to [this template](https://www.kaggle.com/ruriarmandhani/mlb-forecasting-ann))

Function (Json => DataFrame)

In [12]:
import tqdm
from datetime import timedelta

def json_to_df(df, column):
    num_rows = len(df)
    
    data_list = []
    for row in tqdm.tqdm(range(num_rows)):
        
        json_data = df.iloc[row][column]
        if str(json_data) != "nan":
            data = pd.read_json(json_data)
            data_list.append(data)
        
    all_data = pd.concat(data_list, axis = 0)
    
    return all_data

In [13]:
train.head()

,Unnamed: 0,date,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,0,20180101,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe...","[{""date"":""2018-01-01"",""teamId"":147,""teamName"":..."
1,1,20180102,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN,NaN,NaN
2,2,20180103,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN,NaN,NaN
3,3,20180104,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN,NaN,NaN
4,4,20180105,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN,NaN,NaN


A 'player_engagement' column contatins 2018-01-01 ~ 2021-04-26 data.

+ There's a day gap between 'date' & 'engagementMetricsDate'^.

In [14]:
player_engagement = json_to_df(train, 'nextDayPlayerEngagement')
player_engagement.insert(0, 'date', pd.to_datetime(player_engagement['engagementMetricsDate'])-\
                                                   timedelta(days=1))
player_engagement['engagementMetricsDate'] = pd.to_datetime(player_engagement['engagementMetricsDate'])
player_engagement.reset_index(drop=True, inplace=True)
print(player_engagement.shape)
player_engagement.head()

100%|██████████| 1212/1212 [00:21<00:00, 57.43it/s]


(2497932, 7)


,date,engagementMetricsDate,playerId,target1,target2,target3,target4
0,2018-01-01,2018-01-02,628317,0.011167,4.474708,0.005168,5.735294
1,2018-01-01,2018-01-02,547989,0.042993,5.593385,0.045033,2.794118
2,2018-01-01,2018-01-02,519317,0.974327,56.177043,13.693746,64.166667
3,2018-01-01,2018-01-02,607625,0.006700,2.675097,0.005168,1.862745
4,2018-01-01,2018-01-02,592547,0.001117,0.632296,0.002953,0.931373


- 'Date' & 'gameDate' mean the same - one will be eliminated later
- Columns associated with followers are not handled yet.

In [15]:
key = 0

if key == 0:

  ##### Auto Ver. ######
  cols_in_train = ['games', 'rosters', 'playerBoxScores', 'teamBoxScores', 'standings', 'events']

  for df in cols_in_train:
    
    globals()['%s' % df] = json_to_df(train, df)
    globals()['%s' % df].insert(0, 'date', pd.to_datetime(globals()['%s' % df]['gameDate']))
    globals()['%s' % df].drop(['gameDate'], axis = 1, inplace = True)
    globals()['%s' % df].reset_index(drop = True, inplace = True)


  ## special cases

  transactions = json_to_df(train, 'transactions')
  # transactions.insert(0, 'date', pd.to_datetime(transactions['gameDate'])) # Already having 'date' column
  transactions['date'] = pd.to_datetime(transactions['date'])
  transactions.reset_index(drop=True, inplace=True)

  award = json_to_df(train, 'awards')
  award.insert(0, 'date', pd.to_datetime(award['awardDate']))
  award['awardDdate'] = pd.to_datetime(award['awardDate'])
  award.reset_index(drop=True, inplace=True)
  
elif key == 1:

  ##### Manual Ver. #####
  games = json_to_df(train, 'games')
  games.insert(0, 'date', pd.to_datetime(games['gameDate']))
  games.drop(['gameDate'], axis = 1, inplace = True)
  games.reset_index(drop=True, inplace=True)

  rosters = json_to_df(train, 'rosters')
  rosters.insert(0, 'date', pd.to_datetime(rosters['gameDate']))
  rosters.drop(['gameDate'], axis = 1, inplace = True)
  rosters.reset_index(drop=True, inplace=True)

  playerBoxScores = json_to_df(train, 'playerBoxScores')
  playerBoxScores.insert(0, 'date', pd.to_datetime(playerBoxScores['gameDate']))
  playerBoxScores.drop(['gameDate'], axis = 1, inplace = True)
  playerBoxScores.reset_index(drop=True, inplace=True)

  teamBoxScores = json_to_df(train, 'teamBoxScores')
  teamBoxScores.insert(0, 'date', pd.to_datetime(teamBoxScores['gameDate']))
  teamBoxScores.drop(['gameDate'], axis = 1, inplace = True)
  teamBoxScores.reset_index(drop=True, inplace=True)

  standings = json_to_df(train, 'standings')
  standings.insert(0, 'date', pd.to_datetime(standings['gameDate']))
  standings.drop(['gameDate'], axis = 1, inplace = True)
  standings.reset_index(drop=True, inplace=True)

  events = json_to_df(train, 'events')
  events.insert(0, 'date', pd.to_datetime(events['gameDate']))
  events.drop(['gameDate'], axis = 1, inplace = True)
  events.reset_index(drop=True, inplace=True)


  ## special cases

  transactions = json_to_df(train, 'transactions')
  # transactions.insert(0, 'date', pd.to_datetime(transactions['gameDate'])) # Already having 'date' column
  transactions['date'] = pd.to_datetime(transactions['date'])
  transactions.reset_index(drop=True, inplace=True)

  award = json_to_df(train, 'awards')
  award.insert(0, 'date', pd.to_datetime(award['awardDate']))
  award['awardDdate'] = pd.to_datetime(award['awardDate'])
  award.reset_index(drop=True, inplace=True)

100%|██████████| 1212/1212 [00:01<00:00, 708.21it/s]


Parsed ,,train'' gonna be saved as pickle (or csv) file to be brought back anytime I want

In [ ]:
train.head()

,Unnamed: 0,date,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,0,20180101,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe...","[{""date"":""2018-01-01"",""teamId"":147,""teamName"":..."
1,1,20180102,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN,NaN,NaN
2,2,20180103,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN,NaN,NaN
3,3,20180104,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN,NaN,NaN
4,4,20180105,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN,NaN,NaN


In [16]:
PARSED_DF = cols_in_train + ['player_engagement', 'transactions', 'award']

for df in PARSED_DF:
    globals()['%s' % df].to_csv('/content/gdrive/My Drive/Colab Notebooks/MLB/{}.csv'.format(df), index = False)

In [ ]:
## 0. player_engagement

In [ ]:
## 1. games


In [ ]:
games

,date,gamePk,gameType,season,gameTimeUTC,resumeDate,resumedFrom,codedGameState,detailedGameState,isTie,gameNumber,doubleHeader,dayNight,scheduledInnings,gamesInSeries,seriesDescription,homeId,homeName,homeAbbrev,homeWins,homeLosses,homeWinPct,homeWinner,homeScore,awayId,awayName,awayAbbrev,awayWins,awayLosses,awayWinPct,awayWinner,awayScore
0,2018-02-21,533782,E,2018,2018-02-21T20:10:00Z,NaN,NaN,F,Final,0.0,1,N,day,7,0.0,Exhibition,109,Arizona Diamondbacks,ARI,1,0,1.000,True,7.0,5035,Arizona State Sun Devils,ASU,0.0,1.0,0.000,False,2.0
1,2018-02-22,534461,E,2018,2018-02-22T18:05:00Z,NaN,NaN,F,Final,0.0,1,N,day,9,0.0,Exhibition,116,Detroit Tigers,DET,1,0,1.000,True,6.0,228,Florida Southern College Mocs,FSC,0.0,1.0,0.000,False,1.0
2,2018-02-22,545334,E,2018,2018-02-22T18:05:00Z,NaN,NaN,F,Final,0.0,1,N,day,9,0.0,Exhibition,143,Philadelphia Phillies,PHI,1,0,1.000,True,6.0,231,University of Tampa Spartans,UT,0.0,1.0,0.000,False,0.0
3,2018-02-22,547295,E,2018,2018-02-22T03:33:00Z,NaN,NaN,F,Final,0.0,2,Y,night,9,0.0,Exhibition,111,Boston Red Sox,BOS,2,0,1.000,True,4.0,227,Boston College Eagles,BC,0.0,1.0,0.000,False,2.0
4,2018-02-22,533784,E,2018,2018-02-22T23:05:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,0.0,Exhibition,142,Minnesota Twins,MIN,1,0,1.000,True,2.0,4864,Minnesota Gophers,UM,0.0,1.0,0.000,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7801,2021-04-26,634343,R,2021,2021-04-26T23:10:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,4.0,Regular Season,139,Tampa Bay Rays,TB,11,12,0.478,False,1.0,133,Oakland Athletics,OAK,15.0,8.0,0.652,True,2.0
7802,2021-04-26,634345,R,2021,2021-04-26T23:45:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,4.0,Regular Season,138,St. Louis Cardinals,STL,11,11,0.500,False,1.0,143,Philadelphia Phillies,PHI,11.0,11.0,0.500,True,2.0
7803,2021-04-26,634382,R,2021,2021-04-27T00:10:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,4.0,Regular Season,117,Houston Astros,HOU,11,11,0.500,True,5.0,136,Seattle Mariners,SEA,13.0,10.0,0.565,False,2.0
7804,2021-04-26,634394,R,2021,2021-04-26T22:10:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,3.0,Regular Season,114,Cleveland Indians,CLE,10,11,0.476,True,5.0,142,Minnesota Twins,MIN,7.0,14.0,0.333,False,3.0


In [ ]:
## 2. rosters

In [ ]:
## 3. playerBoxScores

In [ ]:
## 4. teamBoxScores

In [ ]:
## 5. standings

In [ ]:
## 6. events

In [ ]:
## 7. transactions

In [ ]:
## 8. awards
award

,date,awardId,awardName,awardDate,awardSeason,playerId,playerName,awardPlayerTeamId,awardDdate
0,2018-01-15,VWLCPOY,VWL Comeback Player of the Year,2018-01-15,2017,150119,Freddy Garcia,699.0,2018-01-15
1,2018-01-15,VWLPOY,VWL Pitcher of the Year,2018-01-15,2017,446861,Guillermo Moscoso,699.0,2018-01-15
2,2018-01-15,VWLMOY,VWL Manager of the Year,2018-01-15,2017,492527,Mike Rojas,695.0,2018-01-15
3,2018-01-15,VWLRLOY,VWL Reliever of the Year,2018-01-15,2017,519246,Andres Santiago,694.0,2018-01-15
4,2018-01-15,VWLOPOY,VWL Offensive Player of the Year,2018-01-15,2017,543874,Jose Vargas,699.0,2018-01-15
...,...,...,...,...,...,...,...,...,...
5902,2021-04-18,ALPOW,AL Player of the Week,2021-04-18,2021,607074,Carlos Rodon,145.0,2021-04-18
5903,2021-04-25,NLPOW,NL Player of the Week,2021-04-25,2021,518516,Madison Bumgarner,109.0,2021-04-25
5904,2021-04-25,NLPOW,NL Player of the Week,2021-04-25,2021,665487,Fernando Tatis Jr.,135.0,2021-04-25
5905,2021-04-25,MLBPLAYOW,MLB Play of the Week,2021-04-25,2021,640449,Clint Frazier,147.0,2021-04-25


## Intergrated data with '____' column shared

In [ ]:
cols_in_train

['games', 'rosters', 'playerBoxScores', 'teamBoxScores', 'standings', 'events']

In [ ]:
games

,date,gamePk,gameType,season,gameDate,gameTimeUTC,resumeDate,resumedFrom,codedGameState,detailedGameState,isTie,gameNumber,doubleHeader,dayNight,scheduledInnings,gamesInSeries,seriesDescription,homeId,homeName,homeAbbrev,homeWins,homeLosses,homeWinPct,homeWinner,homeScore,awayId,awayName,awayAbbrev,awayWins,awayLosses,awayWinPct,awayWinner,awayScore
0,2018-02-21,533782,E,2018,2018-02-21,2018-02-21T20:10:00Z,NaN,NaN,F,Final,0.0,1,N,day,7,0.0,Exhibition,109,Arizona Diamondbacks,ARI,1,0,1.000,True,7.0,5035,Arizona State Sun Devils,ASU,0.0,1.0,0.000,False,2.0
1,2018-02-22,534461,E,2018,2018-02-22,2018-02-22T18:05:00Z,NaN,NaN,F,Final,0.0,1,N,day,9,0.0,Exhibition,116,Detroit Tigers,DET,1,0,1.000,True,6.0,228,Florida Southern College Mocs,FSC,0.0,1.0,0.000,False,1.0
2,2018-02-22,545334,E,2018,2018-02-22,2018-02-22T18:05:00Z,NaN,NaN,F,Final,0.0,1,N,day,9,0.0,Exhibition,143,Philadelphia Phillies,PHI,1,0,1.000,True,6.0,231,University of Tampa Spartans,UT,0.0,1.0,0.000,False,0.0
3,2018-02-22,547295,E,2018,2018-02-22,2018-02-22T03:33:00Z,NaN,NaN,F,Final,0.0,2,Y,night,9,0.0,Exhibition,111,Boston Red Sox,BOS,2,0,1.000,True,4.0,227,Boston College Eagles,BC,0.0,1.0,0.000,False,2.0
4,2018-02-22,533784,E,2018,2018-02-22,2018-02-22T23:05:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,0.0,Exhibition,142,Minnesota Twins,MIN,1,0,1.000,True,2.0,4864,Minnesota Gophers,UM,0.0,1.0,0.000,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7801,2021-04-26,634343,R,2021,2021-04-26,2021-04-26T23:10:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,4.0,Regular Season,139,Tampa Bay Rays,TB,11,12,0.478,False,1.0,133,Oakland Athletics,OAK,15.0,8.0,0.652,True,2.0
7802,2021-04-26,634345,R,2021,2021-04-26,2021-04-26T23:45:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,4.0,Regular Season,138,St. Louis Cardinals,STL,11,11,0.500,False,1.0,143,Philadelphia Phillies,PHI,11.0,11.0,0.500,True,2.0
7803,2021-04-26,634382,R,2021,2021-04-26,2021-04-27T00:10:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,4.0,Regular Season,117,Houston Astros,HOU,11,11,0.500,True,5.0,136,Seattle Mariners,SEA,13.0,10.0,0.565,False,2.0
7804,2021-04-26,634394,R,2021,2021-04-26,2021-04-26T22:10:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,3.0,Regular Season,114,Cleveland Indians,CLE,10,11,0.476,True,5.0,142,Minnesota Twins,MIN,7.0,14.0,0.333,False,3.0


In [ ]:
transactions # date

,transactionId,playerId,playerName,date,fromTeamId,fromTeamName,toTeamId,toTeamName,effectiveDate,resolutionDate,typeCode,typeDesc,description
0,340732,547348.0,C.C. Lee,2018-01-01,NaN,NaN,119,Los Angeles Dodgers,2018-01-01,2018-01-01,SFA,Signed as Free Agent,Los Angeles Dodgers signed free agent RHP C.C....
1,339458,621173.0,Dylan Baker,2018-01-02,158.0,Milwaukee Brewers,119,Los Angeles Dodgers,2018-03-20,None,TR,Trade,Milwaukee Brewers traded RHP Dylan Baker to Lo...
2,357292,678876.0,Angel Rojas,2018-01-02,NaN,None,147,New York Yankees,2018-01-02,2018-01-02,SFA,Signed as Free Agent,New York Yankees signed free agent SS Angel Ro...
3,341123,607054.0,Jace Peterson,2018-01-02,NaN,None,147,New York Yankees,2018-01-02,2018-01-02,SFA,Signed as Free Agent,New York Yankees signed free agent 2B Jace Pet...
4,339458,NaN,None,2018-01-02,119.0,Los Angeles Dodgers,158,Milwaukee Brewers,2018-01-02,None,TR,Trade,Milwaukee Brewers traded RHP Dylan Baker to Lo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40907,480395,542960.0,Brad Brach,2021-04-26,118.0,Kansas City Royals,3300,Royals Alternate Training Site,2021-04-26,None,OUT,Outrighted,Kansas City Royals sent RHP Brad Brach outrigh...
40908,479900,600524.0,Renato Nunez,2021-04-26,116.0,Detroit Tigers,3302,Tigers Alternate Training Site,2021-04-26,None,OUT,Outrighted,Detroit Tigers sent 3B Renato Nunez outright t...
40909,479905,624503.0,Nick Gordon,2021-04-26,142.0,Minnesota Twins,3304,Twins Alternate Training Site,2021-04-26,None,OPT,Optioned,Minnesota Twins optioned SS Nick Gordon to Twi...
40910,479880,680692.0,Jonathan Stiever,2021-04-26,145.0,Chicago White Sox,3306,White Sox Alternate Training Site,2021-04-26,None,OPT,Optioned,Chicago White Sox optioned RHP Jonathan Stieve...


In [ ]:
awards # awardDate

,date,awardId,awardName,awardDate,awardSeason,playerId,playerName,awardPlayerTeamId,awardDdate
0,2018-01-15,VWLCPOY,VWL Comeback Player of the Year,2018-01-15,2017,150119,Freddy Garcia,699.0,2018-01-15
1,2018-01-15,VWLPOY,VWL Pitcher of the Year,2018-01-15,2017,446861,Guillermo Moscoso,699.0,2018-01-15
2,2018-01-15,VWLMOY,VWL Manager of the Year,2018-01-15,2017,492527,Mike Rojas,695.0,2018-01-15
3,2018-01-15,VWLRLOY,VWL Reliever of the Year,2018-01-15,2017,519246,Andres Santiago,694.0,2018-01-15
4,2018-01-15,VWLOPOY,VWL Offensive Player of the Year,2018-01-15,2017,543874,Jose Vargas,699.0,2018-01-15
...,...,...,...,...,...,...,...,...,...
5902,2021-04-18,ALPOW,AL Player of the Week,2021-04-18,2021,607074,Carlos Rodon,145.0,2021-04-18
5903,2021-04-25,NLPOW,NL Player of the Week,2021-04-25,2021,518516,Madison Bumgarner,109.0,2021-04-25
5904,2021-04-25,NLPOW,NL Player of the Week,2021-04-25,2021,665487,Fernando Tatis Jr.,135.0,2021-04-25
5905,2021-04-25,MLBPLAYOW,MLB Play of the Week,2021-04-25,2021,640449,Clint Frazier,147.0,2021-04-25


# Modeling

[MultiOutputRegression](https://stackoverflow.com/questions/39540123/muti-output-regression-in-xgboost)


[MultiOutputRegression(Scikit-learn)](https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputRegressor.html)

In [ ]:
https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputRegressor.html

SyntaxError: ignored

# Evaluation

[Nice template](https://www.kaggle.com/ryanholbrook/getting-started-with-mlb-player-digital-engagement)

In [ ]:
import mlb
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df['target1'] = 100 #make predictions here
    env.predict(sample_prediction_df)

,date,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,20210426,"[{""gamePk"":634374,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-26"",""t...","[{""home"":1,""gamePk"":634377,""gameDate"":""2021-04...","[{""home"":1,""teamId"":139,""gamePk"":634343,""gameD...","[{""transactionId"":480386,""playerId"":543685,""pl...","[{""season"":2021,""gameDate"":""2021-04-26"",""divis...",NaN,"[{""gamePk"":634433,""gameDate"":""2021-04-26"",""gam...",NaN,NaN
1,20210427,"[{""gamePk"":634318,""gameType"":""R"",""season"":2021...","[{""playerId"":443558,""gameDate"":""2021-04-27"",""t...","[{""home"":1,""gamePk"":634320,""gameDate"":""2021-04...","[{""home"":1,""teamId"":117,""gamePk"":634333,""gameD...","[{""transactionId"":480456,""playerId"":642162,""pl...","[{""season"":2021,""gameDate"":""2021-04-27"",""divis...",NaN,"[{""gamePk"":634332,""gameDate"":""2021-04-27"",""gam...",NaN,NaN
2,20210428,"[{""gamePk"":634309,""gameType"":""R"",""season"":2021...","[{""playerId"":429722,""gameDate"":""2021-04-28"",""t...","[{""home"":1,""gamePk"":634310,""gameDate"":""2021-04...","[{""home"":0,""teamId"":111,""gamePk"":634310,""gameD...","[{""transactionId"":480728,""playerId"":545358,""pl...","[{""season"":2021,""gameDate"":""2021-04-28"",""divis...",NaN,"[{""gamePk"":634317,""gameDate"":""2021-04-28"",""gam...",NaN,NaN
3,20210429,"[{""gamePk"":634330,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-29"",""t...","[{""home"":1,""gamePk"":634330,""gameDate"":""2021-04...","[{""home"":0,""teamId"":119,""gamePk"":634346,""gameD...","[{""transactionId"":480993,""playerId"":606965,""pl...","[{""season"":2021,""gameDate"":""2021-04-29"",""divis...",NaN,"[{""gamePk"":634346,""gameDate"":""2021-04-29"",""gam...",NaN,NaN
4,20210430,"[{""gamePk"":634287,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-30"",""t...","[{""home"":1,""gamePk"":634305,""gameDate"":""2021-04...","[{""home"":1,""teamId"":135,""gamePk"":634303,""gameD...",NaN,"[{""season"":2021,""gameDate"":""2021-04-30"",""divis...","[{""awardId"":""NLRRELMON"",""awardName"":""NL Reliev...","[{""gamePk"":634327,""gameDate"":""2021-04-30"",""gam...",NaN,NaN
